In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-data/train.csv
/kaggle/input/titanic-data/test.csv
/kaggle/input/titanic-data/gender_submission.csv


In [2]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

/tmp/ipykernel_19/2889067548.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


# refactor the data

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic-data/test.csv")
train_data = pd.read_csv("/kaggle/input/titanic-data/train.csv")

columns_to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]

train_labels = train_data["Survived"]
train_data.drop(columns_to_drop, axis=1, inplace=True)
train_data.drop("Survived", axis=1, inplace=True)

test_data.drop(columns_to_drop, axis=1, inplace=True)

train_data["Sex"] = train_data["Sex"].replace({"male":0, "female":1})
train_data["Embarked"] = train_data["Embarked"].replace({"S":0, "C":1, "Q":2})
test_data["Sex"] = test_data["Sex"].replace({"male":0, "female":1})
test_data["Embarked"] = test_data["Embarked"].replace({"S":0, "C":1, "Q":2})

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)
train_data


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,0.0
1,1,1,38.0,1,0,71.2833,1.0
2,3,1,26.0,0,0,7.9250,0.0
3,1,1,35.0,1,0,53.1000,0.0
4,3,0,35.0,0,0,8.0500,0.0
...,...,...,...,...,...,...,...
885,3,1,39.0,0,5,29.1250,2.0
886,2,0,27.0,0,0,13.0000,0.0
887,1,1,19.0,0,0,30.0000,0.0
889,1,0,26.0,0,0,30.0000,1.0


# Normalize the data

In [4]:
train_dataNorm = train_data/np.max(train_data)
test_dataNorm = test_data/np.max(test_data)

test_dataNorm

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.005856,0.000000,0.067340,0.000000,0.000000,0.015282,0.003904
1,0.005856,0.001952,0.091738,0.001952,0.000000,0.013663,0.000000
2,0.003904,0.000000,0.121016,0.000000,0.000000,0.018909,0.003904
3,0.005856,0.000000,0.052700,0.000000,0.000000,0.016908,0.000000
4,0.005856,0.001952,0.042941,0.001952,0.001952,0.023984,0.000000
...,...,...,...,...,...,...,...
409,0.005856,0.001952,0.005856,0.001952,0.001952,0.026887,0.000000
411,0.001952,0.001952,0.072219,0.001952,0.000000,0.175668,0.003904
412,0.005856,0.001952,0.054652,0.000000,0.000000,0.015176,0.000000
414,0.001952,0.001952,0.076123,0.000000,0.000000,0.212559,0.001952
